In [143]:
import os

In [144]:
import pandas as pd

In [145]:
fnames = [x for x in os.listdir() if x.endswith('csv')]

In [146]:
fnames

['config-xor-bad-1-results.csv',
 'config-xor-bad-2-missing-results.csv',
 'config-xor-bad-2-results.csv',
 'config-xor-bad-3-missing-2-results.csv',
 'config-xor-bad-3-results.csv',
 'config-xor-bad-4-one-weight-combs-results.csv',
 'config-xor-bad-5-one-weight-combs-results.csv']

In [147]:
dfs = {}
for name in fnames:
    dfs[name] = pd.read_csv(name)

### desc

- 'config-xor-bad-4-one-weight-combs-results.csv' prop: [50, -15]
- 'config-xor-bad-5-one-weight-combs-results.csv' prop: [7, -15]

In [148]:
dfs['config-xor-bad-4-one-weight-combs-results.csv']

,exp_id,avg_acc_before,avg_acc_after,time,Props,Heuristic,NumConstraints,Threshold,NNHidden,Params,Free,WeightsConfig,extra_params
0,1,unsat,unsat,0.2911,1,SAMPLES,450,1,[4],22,1,"{'select_weight': [[1, 1, 1]]}",NaN
1,2,unsat,unsat,0.4585,1,SAMPLES,450,250,[4],22,1,"{'select_weight': [[1, 1, 1]]}",NaN
2,3,unsat,unsat,0.4465,1,SAMPLES,450,400,[4],22,1,"{'select_weight': [[1, 1, 1]]}",NaN
3,4,unsat,unsat,0.3036,1,SAMPLES,450,1,[4],22,1,"{'select_weight': [[1, 1, 2]]}",NaN
4,5,unsat,unsat,0.6235,1,SAMPLES,450,250,[4],22,1,"{'select_weight': [[1, 1, 2]]}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,150,unknown,unknown,timeout,1,VORONOI,153,2,[4],22,1,"{'select_weight': [[2, 2, 3]]}","{'limit_points': 200, 'resample': True, 'rando..."
150,151,unsat,unsat,1.7127,1,VORONOI,153,1,[4],22,1,"{'select_weight': [[2, 2, 4]]}","{'limit_points': 200, 'resample': True, 'rando..."
151,152,unknown,unknown,timeout,1,VORONOI,153,2,[4],22,1,"{'select_weight': [[2, 2, 4]]}","{'limit_points': 200, 'resample': True, 'rando..."
152,153,0.9950846531949754,0.375204806116876,57.7394,1,VORONOI,153,1,[4],22,1,"{'select_bias': [[2, 2]]}","{'limit_points': 200, 'resample': True, 'rando..."


In [149]:
def agg_groups(groups):
    res = []
    for name, group in groups:
        print(name)
        _curr = {
            'Heuristic': name,
            'N': len(group),
            'SoftConstraints': pd.to_numeric(group['NumConstraints'], errors='coerce').unique(),
            'Thresholds': pd.to_numeric(group['Threshold'], errors='coerce').unique(),
            'AccBefore': pd.to_numeric(group['avg_acc_before'], errors='coerce').mean(),
            'MaxAcc': pd.to_numeric(group['avg_acc_after'], errors='coerce').max(),
            'MinAcc': pd.to_numeric(group['avg_acc_after'], errors='coerce').min(),
            'AvgAcc': pd.to_numeric(group['avg_acc_after'], errors='coerce').mean(),
            'SAT': pd.to_numeric(group['avg_acc_after'], errors='coerce').notnull().sum(),
            'UNSAT': (group['avg_acc_after']=='unsat').values.sum(),
            'Timeout': (group['time']=='timeout').values.sum(),
            'TotalSolverTime': int(pd.to_numeric(group['time'], errors='coerce').sum() + (group['time']=='timeout').values.sum()*600),
        }
        res.append(_curr)
        
    return pd.DataFrame(res)

In [150]:
float_format = "{0:.4%}"

### 1-1-weight comb-1st-property

In [151]:
a1 = agg_groups(dfs['config-xor-bad-4-one-weight-combs-results.csv'].groupby('Heuristic'))
a1

GRID
SAMPLES
VORONOI


,Heuristic,N,SoftConstraints,Thresholds,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,TotalSolverTime
0,GRID,44,[100],"[1, 2]",0.995085,0.85964,0.375205,0.601614,9,26,9,6397
1,SAMPLES,66,[450],"[1, 250, 400]",0.995085,0.86947,0.375205,0.718988,15,51,0,32
2,VORONOI,44,[153],"[1, 2]",0.995085,0.85964,0.375205,0.559803,9,13,22,13994


In [152]:
print(a1.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{lrllrrrrrrrr}
\toprule
Heuristic &  N & SoftConstraints &    Thresholds &  AccBefore &   MaxAcc &   MinAcc &   AvgAcc &  SAT &  UNSAT &  Timeout &  TotalSolverTime \\
\midrule
     GRID & 44 &           [100] &        [1, 2] &   99.5085\% & 85.9640\% & 37.5205\% & 60.1614\% &    9 &     26 &        9 &             6397 \\
  SAMPLES & 66 &           [450] & [1, 250, 400] &   99.5085\% & 86.9470\% & 37.5205\% & 71.8988\% &   15 &     51 &        0 &               32 \\
  VORONOI & 44 &           [153] &        [1, 2] &   99.5085\% & 85.9640\% & 37.5205\% & 55.9803\% &    9 &     13 &       22 &            13994 \\
\bottomrule
\end{tabular}



In [153]:
#a1.to_csv('1-1-weight comb-1st-property.csv', index=False)

### 2-1-weight comb-2nd-property

In [154]:
a2 = agg_groups(dfs['config-xor-bad-5-one-weight-combs-results.csv'].groupby('Heuristic'))
a2

GRID
SAMPLES
VORONOI


,Heuristic,N,SoftConstraints,Thresholds,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,TotalSolverTime
0,GRID,44,[100],"[1, 2]",0.995085,0.989896,0.375205,0.655980,10,24,10,6234
1,SAMPLES,66,[450],"[1, 250, 400]",0.995085,0.989896,0.375205,0.836997,24,42,0,32
2,VORONOI,44,[153],"[1, 2]",0.995085,0.963681,0.375205,0.606827,10,12,22,13843


In [155]:
print(a2.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{lrllrrrrrrrr}
\toprule
Heuristic &  N & SoftConstraints &    Thresholds &  AccBefore &   MaxAcc &   MinAcc &   AvgAcc &  SAT &  UNSAT &  Timeout &  TotalSolverTime \\
\midrule
     GRID & 44 &           [100] &        [1, 2] &   99.5085\% & 98.9896\% & 37.5205\% & 65.5980\% &   10 &     24 &       10 &             6234 \\
  SAMPLES & 66 &           [450] & [1, 250, 400] &   99.5085\% & 98.9896\% & 37.5205\% & 83.6997\% &   24 &     42 &        0 &               32 \\
  VORONOI & 44 &           [153] &        [1, 2] &   99.5085\% & 96.3681\% & 37.5205\% & 60.6827\% &   10 &     12 &       22 &            13843 \\
\bottomrule
\end{tabular}



In [156]:
#a2.to_csv('2-1-weight comb-2nd-property.csv', index=False)

### 3-Selecting multiple weights-neurons-or-layers-2nd-property

In [157]:
three_combined = dfs['config-xor-bad-3-results.csv'].append(dfs['config-xor-bad-3-missing-2-results.csv'])

In [158]:
a3 = agg_groups(three_combined.groupby('Heuristic'))
a3

GRID
SAMPLES
VORONOI


,Heuristic,N,SoftConstraints,Thresholds,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,TotalSolverTime
0,GRID,55,"[100, 441]","[1, 2]",0.995085,0.593392,0.375205,0.441168,9,27,19,11522
1,SAMPLES,56,[450],"[1, 250, 400, 450]",0.995085,0.980339,0.375205,0.837520,13,36,7,4339
2,VORONOI,56,"[153, 82]","[1, 2]",0.995085,0.802567,0.373020,0.479465,10,18,28,16869


In [159]:
print(a3.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{lrllrrrrrrrr}
\toprule
Heuristic &  N & SoftConstraints &         Thresholds &  AccBefore &   MaxAcc &   MinAcc &   AvgAcc &  SAT &  UNSAT &  Timeout &  TotalSolverTime \\
\midrule
     GRID & 55 &      [100, 441] &             [1, 2] &   99.5085\% & 59.3392\% & 37.5205\% & 44.1168\% &    9 &     27 &       19 &            11522 \\
  SAMPLES & 56 &           [450] & [1, 250, 400, 450] &   99.5085\% & 98.0339\% & 37.5205\% & 83.7520\% &   13 &     36 &        7 &             4339 \\
  VORONOI & 56 &       [153, 82] &             [1, 2] &   99.5085\% & 80.2567\% & 37.3020\% & 47.9465\% &   10 &     18 &       28 &            16869 \\
\bottomrule
\end{tabular}



In [160]:
#a3.to_csv('3-Selecting multiple weights-neurons-or-layers-2nd-property.csv', index=False)

#### 4-Selecting-multiple-weights-neurons-or layers-1st-property 

In [161]:
two_combined = dfs['config-xor-bad-2-results.csv'].append(dfs['config-xor-bad-2-missing-results.csv'])

In [162]:
one_combined = dfs['config-xor-bad-1-results.csv'].append(two_combined)

In [163]:
a4 = agg_groups(one_combined.groupby('Heuristic'))
a4

GRID
SAMPLES
VORONOI


,Heuristic,N,SoftConstraints,Thresholds,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,TotalSolverTime
0,GRID,38,"[100, 400]","[1, 2, 10]",0.995085,0.859640,0.375205,0.609448,15,0,23,13812
1,SAMPLES,36,[450],"[1, 250, 400]",0.995085,0.986073,0.375205,0.844578,32,4,0,171
2,VORONOI,39,"[268, 250, 153]","[1, 2, 10]",0.995085,0.862370,0.375205,0.609030,15,0,24,14424


In [164]:
print(a4.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{lrllrrrrrrrr}
\toprule
Heuristic &  N & SoftConstraints &    Thresholds &  AccBefore &   MaxAcc &   MinAcc &   AvgAcc &  SAT &  UNSAT &  Timeout &  TotalSolverTime \\
\midrule
     GRID & 38 &      [100, 400] &    [1, 2, 10] &   99.5085\% & 85.9640\% & 37.5205\% & 60.9448\% &   15 &      0 &       23 &            13812 \\
  SAMPLES & 36 &           [450] & [1, 250, 400] &   99.5085\% & 98.6073\% & 37.5205\% & 84.4578\% &   32 &      4 &        0 &              171 \\
  VORONOI & 39 & [268, 250, 153] &    [1, 2, 10] &   99.5085\% & 86.2370\% & 37.5205\% & 60.9030\% &   15 &      0 &       24 &            14424 \\
\bottomrule
\end{tabular}



In [165]:
#a4.to_csv('4-Selecting-multiple-weights-neurons-or layers-1st-property.csv', index=False)#

### Utility - combine all results

In [205]:
alldfs = pd.DataFrame()

In [206]:
for name, df in dfs.items():
    df['source'] = name.split('.')[0]
    alldfs = alldfs.append(df)

In [207]:
alldfs = alldfs.set_index(['source', 'exp_id'])

In [208]:
alldfs.head()

avg_acc_before avg_acc_after     time  Props  \
source                   exp_id                                                
config-xor-bad-1-results 1            0.995085      0.650464   0.2746      1   
                         2            0.995085      0.981704  30.4846      1   
                         3            0.995085      0.981704   3.0699      1   
                         4            0.995085      0.649918   0.3767      1   
                         5            0.995085      0.979792   4.3493      1   

                                Heuristic  NumConstraints  Threshold NNHidden  \
source                   exp_id                                                 
config-xor-bad-1-results 1        SAMPLES             450          1      [4]   
                         2        SAMPLES             450        250      [4]   
                         3        SAMPLES             450        400      [4]   
                         4        SAMPLES             450          1      [4]   
                         5        SAMPLES             450        250      [4]   

                                 Params  Free                WeightsConfig  \
source                   exp_id                                              
config-xor-bad-1-results 1           22     5  {'select_neuron': [[2, 1]]}   
                         2           22     5  {'select_neuron': [[2, 1]]}   
                         3           22     5  {'select_neuron': [[2, 1]]}   
                         4           22     5  {'select_neuron': [[2, 2]]}   
                         5           22     5  {'select_neuron': [[2, 2]]}   

                                extra_params  
source                   exp_id               
config-xor-bad-1-results 1               NaN  
                         2               NaN  
                         3               NaN  
                         4               NaN  
                         5               NaN

In [210]:
alldfs.to_csv('combined.csv')